In [ ]:
# UDP Broadcast Server (No netifaces dependency)
# Run this in one Jupyter notebook

import socket
import time
import json
import datetime

# Create UDP socket
server_socket = socket.socket(socket.AF_INET, socket.SOCK_DGRAM, socket.IPPROTO_UDP)

# Enable broadcasting mode
server_socket.setsockopt(socket.SOL_SOCKET, socket.SO_BROADCAST, 1)

# Set a timeout so the socket will not block indefinitely
server_socket.settimeout(0.2)

# Bind socket to a specific port (optional but recommended)
# Using 0.0.0.0 to listen on all available interfaces
server_socket.bind(('0.0.0.0', 0))  # Use port 0 to get a random available port

# Get the assigned port
server_port = server_socket.getsockname()[1]
print(f"Server running on port {server_port}")

# Use the standard broadcast address
# Note: This will broadcast to the local subnet only
broadcast_address = '255.255.255.255'
print(f"Broadcasting to {broadcast_address}")

# Store clients who have responded (for informational purposes)
clients = set()

# Define the port clients will listen on
client_port = 37020

try:
    # Server state (can be modified as needed)
    states = ["ACTIVE", "STANDBY", "MAINTENANCE", "ERROR"]
    current_state_index = 0
    
    message_count = 0
    
    print("Server starting... Press Ctrl+C to stop")
    
    while True:
        # Create message with timestamp and current state
        timestamp = datetime.datetime.now().isoformat()
        current_state = states[current_state_index]
        
        message = {
            "timestamp": timestamp,
            "state": current_state,
            "message_id": message_count
        }
        
        # Convert message to JSON string and encode to bytes
        message_bytes = json.dumps(message).encode('utf-8')
        
        # Broadcast message to all clients
        server_socket.sendto(message_bytes, (broadcast_address, client_port))
        
        print(f"Broadcast message: {message}")
        message_count += 1
        
        # Cycle through states for demonstration purposes
        current_state_index = (current_state_index + 1) % len(states)
        
        # Wait to receive any responses from clients (optional)
        try:
            while True:
                data, addr = server_socket.recvfrom(1024)
                client_message = data.decode('utf-8')
                client_ip = addr[0]
                
                if client_ip not in clients:
                    clients.add(client_ip)
                    print(f"New client connected: {client_ip}, total clients: {len(clients)}")
                
                print(f"Response from {client_ip}: {client_message}")
        except socket.timeout:
            # Expected timeout, continue to next broadcast
            pass
        
        # Wait before sending next message
        time.sleep(5)  # Adjust delay as needed

except KeyboardInterrupt:
    print("\nServer shutting down...")
finally:
    server_socket.close()
    print("Server stopped")